## Sistem Rekomendasi dengan content-based filtering

### import library pendukung

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px 

from sklearn.feature_extraction.text import TfidfVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.metrics.pairwise import cosine_similarity

from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path

tv = TfidfVectorizer(max_features=5000)
stem = StemmerFactory().create_stemmer()
stopword = StopWordRemoverFactory().create_stop_word_remover()

In [2]:
all_turis = pd.read_csv("C:/Users/dinar/tempat_wisata.csv")
all_turis

,Unnamed: 0,Place_Id,Place_Ratings,Place_Name,Description,Category,City,Price
0,0,1,3.722222,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000
1,1,2,2.840000,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0
2,2,3,2.526316,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000
3,3,4,2.857143,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000
4,4,5,3.520000,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000
...,...,...,...,...,...,...,...,...
432,432,433,3.304348,Museum Mpu Tantular,Museum Negeri Mpu Tantular adalah sebuah museu...,Budaya,Surabaya,2000
433,433,434,2.400000,Taman Bungkul,Taman Bungkul adalah taman wisata kota yang te...,Taman Hiburan,Surabaya,0
434,434,435,3.000000,Taman Air Mancur Menari Kenjeran,Air mancur menari atau dancing fountain juga a...,Taman Hiburan,Surabaya,0
435,435,436,3.090909,Taman Flora Bratang Surabaya,Taman Flora adalah salah satu taman kota di Su...,Taman Hiburan,Surabaya,0


### Modeling

In [4]:
def preprocessing(data):
    data = data.lower()
    data = stem.stem(data)
    data = stopword.remove(data)
    return data

In [56]:
content_based_filtering = all_turis.copy()
content_based_filtering['Tag'] = content_based_filtering['Description'] + ' ' + content_based_filtering['Category']
content_based_filtering.drop(['Price','Place_Ratings','Description','Category',],axis=1,inplace=True)
content_based_filtering

,Unnamed: 0,Place_Id,Place_Name,City,Tag
0,0,1,Monumen Nasional,Jakarta,Monumen Nasional atau yang populer disingkat d...
1,1,2,Kota Tua,Jakarta,"Kota tua di Jakarta, yang juga bernama Kota Tu..."
2,2,3,Dunia Fantasi,Jakarta,Dunia Fantasi atau disebut juga Dufan adalah t...
3,3,4,Taman Mini Indonesia Indah (TMII),Jakarta,Taman Mini Indonesia Indah merupakan suatu kaw...
4,4,5,Atlantis Water Adventure,Jakarta,Atlantis Water Adventure atau dikenal dengan A...
...,...,...,...,...,...
432,432,433,Museum Mpu Tantular,Surabaya,Museum Negeri Mpu Tantular adalah sebuah museu...
433,433,434,Taman Bungkul,Surabaya,Taman Bungkul adalah taman wisata kota yang te...
434,434,435,Taman Air Mancur Menari Kenjeran,Surabaya,Air mancur menari atau dancing fountain juga a...
435,435,436,Taman Flora Bratang Surabaya,Surabaya,Taman Flora adalah salah satu taman kota di Su...


In [41]:
content_based_filtering.Tag = content_based_filtering.Tag.apply(preprocessing)
content_based_filtering

,Unnamed: 0,Place_Id,Place_Name,City,Tag
0,0,1,Monumen Nasional,Jakarta,monumen nasional populer singkat monas tugu mo...
1,1,2,Kota Tua,Jakarta,kota tua jakarta juga nama kota tua pusat alun...
2,2,3,Dunia Fantasi,Jakarta,dunia fantasi sebut dufan tempat hibur letak k...
3,3,4,Taman Mini Indonesia Indah (TMII),Jakarta,taman mini indonesia indah rupa suatu kawasan ...
4,4,5,Atlantis Water Adventure,Jakarta,atlantis water adventure kenal atlantis ancol ...
...,...,...,...,...,...
432,432,433,Museum Mpu Tantular,Surabaya,museum negeri mpu tantular buah museum negeri ...
433,433,434,Taman Bungkul,Surabaya,taman bungkul taman wisata kota letak pusat ko...
434,434,435,Taman Air Mancur Menari Kenjeran,Surabaya,air mancur tari dancing fountain ada kawasan j...
435,435,436,Taman Flora Bratang Surabaya,Surabaya,taman flora salah satu taman kota surabaya rup...


In [42]:
vec = tv.fit_transform(content_based_filtering.Tag).toarray()
vec

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [43]:
similarity = cosine_similarity(vec)
similarity[0][1:10]

array([0.06078477, 0.08962216, 0.04623029, 0.01675101, 0.04939059,
       0.03379502, 0.00983281, 0.03406665, 0.03878188])

In [54]:
def recommend_destination_by_content_based_filtering(nama_tempat_wisata):
    nama_tempat__wisata_index = content_based_filtering[content_based_filtering['Place_Name']==nama_tempat_wisata].index[0]
    distancess = similarity[nama_tempat__wisata_index]
    nama_tempat_list = sorted(list(enumerate(distancess)),key=lambda x: x[1],reverse=True)[1:10]
    
    recommended_nama_tempat = []
    for i in nama_tempat_list:
        recommended_nama_tempat.append([content_based_filtering.iloc[i[0]].Place_Name]+[i[1]])
        # print(nama_tempats.iloc[i[0]].original_title)
        
    return recommended_nama_tempat

### hasil

In [55]:
recommend_destination_by_content_based_filtering('Monumen Nasional')

[['Monumen Bandung Lautan Api', 0.2832718606611189],
 ['Monumen Selamat Datang', 0.2770463117990556],
 ['Monumen Perjuangan Rakyat Jawa Barat', 0.25514587875049966],
 ['Tugu Muda Semarang', 0.2215698275635468],
 ['Monumen Bambu Runcing Surabaya', 0.20033709072369268],
 ['Monumen Tugu Pahlawan', 0.18529694176337738],
 ['Monumen Sanapati', 0.1781690714321966],
 ['Monumen Yogya Kembali', 0.17734907513042333],
 ['Monumen Palagan Ambarawa', 0.16746361086475367]]

### kesimpulan
model ini memberikan rekomendasi tempat wisata lainnya yang serupa, jika kita mengetikkan tempat wisata sesuai tujuan kita. model ini cocok untuk pencarian tempat wisata yang sudah spesifik ingin pengguna tuju, tapi ingin melihat rekomendasi tempat wisata serupa dengan yang dituju. content-based filtering kurang bagus jika kita ingin membuat model pencarian tempat wisata yang belum spesifik diketahui, atau pengguna masih bingung ingin liburan ke tempat wisata yang mana.